In [27]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import time
import os

In [ ]:
df = pd.read_csv('./rawData/Foursquare_global/dataset_TIST2015_Checkins.txt', sep='\t', names=['uid', 'pid', 't', 'offset'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33263633 entries, 0 to 33263632
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   uid     int64 
 1   pid     object
 2   t       object
 3   offset  int64 
dtypes: int64(2), object(2)
memory usage: 1015.1+ MB


In [ ]:
df['t_len'] = df.t.apply(lambda x: len(x))
f_df = df[df['t_len']==30]
len(df), len(f_df)

(33263633, 33263631)

In [ ]:
def tzo_convert(tzo_to_convert):
    return  datetime.timedelta(hours=tzo_to_convert/60)
f_df['check-in_time'] = f_df['t'].parallel_apply(lambda x: datetime.datetime.strptime(x, "%a %b %d %H:%M:%S +0000 %Y"))
f_df['check-in_time'] = f_df.parallel_apply(lambda x: str(x['check-in_time'] + tzo_convert(x['offset'])), axis=1)
f_df['day'] = f_df['check-in_time'].parallel_apply(lambda x : x[:10])
f_df = f_df.rename(columns = {'uid' : 'user', 'pid' : 'location_id'})
f_df.to_csv('./rawData/Foursquare_global/check_in.csv')
f_df.head()

In [ ]:
df = pd.read_csv('./rawData/Foursquare_global/check_in.csv')
df.head()

,user,location_id,t,offset,t_len,check-in_time,day
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240,30,2012-04-03 22:00:06,2012-04-03
1,190571,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180,30,2012-04-03 21:00:07,2012-04-03
2,221021,4a85b1b3f964a520eefe1fe3,Tue Apr 03 18:00:08 +0000 2012,-240,30,2012-04-03 14:00:08,2012-04-03
3,66981,4b4606f2f964a520751426e3,Tue Apr 03 18:00:08 +0000 2012,-300,30,2012-04-03 13:00:08,2012-04-03
4,21010,4c2b4e8a9a559c74832f0de2,Tue Apr 03 18:00:09 +0000 2012,240,30,2012-04-03 22:00:09,2012-04-03


In [ ]:
p_df = pd.read_csv("./rawData/Foursquare_global/dataset_TIST2015_POIs.txt", sep='\t', names=['location_id', 'p_lat', 'p_lon', 'p_type', 'countrycode'])
check_gps_df = pd.merge(df, p_df, on = 'location_id')
print(len(df), len(p_df), len(check_gps_df))
check_gps_df.head()

33263631 3680126 33263631


,user,location_id,t,offset,t_len,check-in_time,day,p_lat,p_lon,p_type,countrycode
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240,30,2012-04-03 22:00:06,2012-04-03,55.696132,37.557842,Comedy Club,RU
1,190571,4b4b87b5f964a5204a9f26e3,Tue Apr 03 18:00:07 +0000 2012,180,30,2012-04-03 21:00:07,2012-04-03,41.029717,28.974420,Other Great Outdoors,TR
2,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 05:49:28 +0000 2012,180,30,2012-04-04 08:49:28,2012-04-04,41.029717,28.974420,Other Great Outdoors,TR
3,227763,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:10:31 +0000 2012,180,30,2012-04-04 15:10:31,2012-04-04,41.029717,28.974420,Other Great Outdoors,TR
4,212251,4b4b87b5f964a5204a9f26e3,Wed Apr 04 12:53:32 +0000 2012,180,30,2012-04-04 15:53:32,2012-04-04,41.029717,28.974420,Other Great Outdoors,TR


In [ ]:
c_df = pd.read_csv("./rawData/Foursquare_global/dataset_TIST2015_Cities.txt", sep='\t', names=['city', 'c_lat', 'c_lon', 'c_countrycode', 'c_country', 'c_type'])
city_tuple = list(tuple(zip(c_df['c_lat'], c_df['c_lon'], c_df['city'])))
c_df.head()

,city,c_lat,c_lon,c_countrycode,c_country,c_type
0,Cuiaba,-15.615000,-56.093004,BR,Brazil,Provincial capital
1,Brasilia,-15.792111,-47.897748,BR,Brazil,National and provincial capital
2,Goiania,-16.727004,-49.255001,BR,Brazil,Provincial capital
3,Campo Grande,-20.450997,-54.615996,BR,Brazil,Provincial capital
4,Puerto Presidente Stroessner,-25.526997,-54.622997,PY,Paraguay,Provincial capital


In [ ]:
import math
def get_city(lat, lon, city_tup):
    min_dist = float('inf')
    for city in city_tup:
        dist = math.sqrt((lat - city[0])**2 + (lon - city[1])**2)
        if dist < min_dist:
            min_dist = dist
            loc_city = city[2]
    return loc_city
check_gps_df['city'] = check_gps_df.parallel_apply(lambda x : get_city(x['p_lat'], x['p_lon'], city_tuple), axis=1)
res_df = pd.merge(check_gps_df, c_df, on = 'city')
res_df.head()

,user,location_id,t,offset,t_len,check-in_time,day,p_lat,p_lon,p_type,countrycode,city,c_lat,c_lon,c_countrycode,c_country,c_type
0,50756,4f5e3a72e4b053fd6a4313f6,Tue Apr 03 18:00:06 +0000 2012,240,30,2012-04-03 22:00:06,2012-04-03,55.696132,37.557842,Comedy Club,RU,Moscow,55.749996,37.700002,RU,Russia,National and provincial capital
1,546,4f43d962e4b080493849087b,Tue Apr 03 18:00:32 +0000 2012,240,30,2012-04-03 22:00:32,2012-04-03,55.720567,37.513970,Home (private),RU,Moscow,55.749996,37.700002,RU,Russia,National and provincial capital
2,546,4f43d962e4b080493849087b,Wed Apr 04 14:31:39 +0000 2012,240,30,2012-04-04 18:31:39,2012-04-04,55.720567,37.513970,Home (private),RU,Moscow,55.749996,37.700002,RU,Russia,National and provincial capital
3,546,4f43d962e4b080493849087b,Wed Apr 04 16:38:14 +0000 2012,240,30,2012-04-04 20:38:14,2012-04-04,55.720567,37.513970,Home (private),RU,Moscow,55.749996,37.700002,RU,Russia,National and provincial capital
4,546,4f43d962e4b080493849087b,Fri Apr 06 13:29:03 +0000 2012,240,30,2012-04-06 17:29:03,2012-04-06,55.720567,37.513970,Home (private),RU,Moscow,55.749996,37.700002,RU,Russia,National and provincial capital


In [ ]:
res_df = res_df.rename(columns = {'countrycode' : 'p_countrycode'})
res_df.to_csv(f'./rawData/Foursquare_global/check_in_merge.csv', index=False)

In [18]:
dir = f'./rawData/Foursquare_global'
sub_dir = 'city_user_day_fix'
Path(f'{dir}/{sub_dir}/res').mkdir(parents=True, exist_ok=True)

df = pd.read_csv(f'{dir}/check_in_merge.csv')
part_df = df[['user', 'day', 'check-in_time']].reset_index().rename(columns={'index':'ori_id'})
part_df['check_in_time'] = pd.to_datetime(part_df['check-in_time'])
part_df['day'] = pd.to_datetime(part_df['day'])
part_df = part_df.sort_values(['user', 'day', 'check-in_time'])
part_df['hour'] = part_df['check-in_time'].apply(lambda x: x.hour)
part_df['minute'] = part_df['check-in_time'].apply(lambda x: x.minute)
hour_mask = part_df['hour'].apply(lambda x: x < 23).astype(int)
hour_offset = part_df['minute'].apply(lambda x: 1 if x > 30 else 0)
minute_offset = part_df['minute'].apply(lambda x: 60 - x if x > 30 else x)
part_df['hour_within_30min'] = part_df['hour'] + hour_offset * hour_mask
part_df['minute_offset'] = minute_offset
part_df[['hour', 'minute', 'hour_within_30min', 'minute_offset']].describe()

,hour,minute,hour_within_30min,minute_offset
count,3.343755e+07,3.343755e+07,3.343755e+07,3.343755e+07
mean,1.420166e+01,2.947353e+01,1.467138e+01,1.496481e+01
std,5.396462e+00,1.735087e+01,5.390045e+00,8.676002e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.100000e+01,1.400000e+01,1.100000e+01,7.000000e+00
50%,1.500000e+01,2.900000e+01,1.500000e+01,1.500000e+01
75%,1.800000e+01,4.500000e+01,1.900000e+01,2.200000e+01
max,2.300000e+01,5.900000e+01,2.300000e+01,3.000000e+01


In [20]:
part_df = part_df.sort_values(['user', 'day', 'hour_within_30min', 'minute_offset'])
hour_df = part_df.drop_duplicates(['user', 'day', 'hour_within_30min'], keep='first').reset_index(drop=True)
hour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28299352 entries, 0 to 28299351
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ori_id             int64         
 1   user               int64         
 2   day                datetime64[ns]
 3   check-in_time      datetime64[ns]
 4   check_in_time      datetime64[ns]
 5   hour               int64         
 6   minute             int64         
 7   hour_within_30min  int64         
 8   minute_offset      int64         
dtypes: datetime64[ns](3), int64(6)
memory usage: 1.9 GB


In [24]:
gb_cnt_sorted = hour_df.groupby(['user', 'day']).count().sort_values(['user', 'day']).reset_index()
gb_cnt_sorted = gb_cnt_sorted.rename(columns={'check-in_time':'user_day_count'})
gb_cnt_sorted[['user', 'day', 'user_day_count']].to_csv(f'{dir}/{sub_dir}/res/gb_cnt_sorted.csv', index=False)
ans_df = df.iloc[hour_df['ori_id'].to_numpy()]
ans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28299352 entries, 11339490 to 22390795
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user           int64  
 1   location_id    object 
 2   t              object 
 3   offset         int64  
 4   t_len          int64  
 5   check-in_time  object 
 6   day            object 
 7   p_lat          float64
 8   p_lon          float64
 9   p_type         object 
 10  p_countrycode  object 
 11  city           object 
 12  c_lat          float64
 13  c_lon          float64
 14  c_countrycode  object 
 15  c_country      object 
 16  c_type         object 
dtypes: float64(4), int64(3), object(10)
memory usage: 3.8+ GB


In [25]:
ans_df['hour'] = hour_df['hour_within_30min']
ans_df['hour_offset'] = hour_df['minute_offset']

/tmp/ipykernel_387466/1737551785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['hour'] = hour_df['hour_within_30min']
/tmp/ipykernel_387466/1737551785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ans_df['hour_offset'] = hour_df['minute_offset']


In [26]:
ans_df.to_csv(f'{dir}/{sub_dir}/res/gb_sorted.csv', index=False)